# import

In [17]:
import openpyxl
import math

# Загрузка данных из .xlsx

In [14]:
default_path = '3302_Kalugina_Sarzhevskiy.xlsx'

path = input('Введите путь к .xlsx таблице (оставьте пустым для загрузки из таблицы по умолчанию): ')

wb = openpyxl.load_workbook(default_path) if path == '' else openpyxl.load_workbook(path)
sh = wb.active

input_data = []

for i in range(1, 301):
    input_data.append(float(sh.cell(row=i, column=1).value))

if path == '':
    print(f"Загружена трасса из {len(input_data)} чисел из файла {default_path}")
else:
    print(f"Загружена трасса из {len(input_data)} чисел из файла {path}")

Введите путь к .xlsx таблице (оставьте пустым для загрузки из таблицы по умолчанию): 
Загружена трасса из 300 из файла 3302_Kalugina_Sarzhevskiy.xlsx


# Оценки мат. ожидания, дисперсии и ср. кв. отклонения

In [29]:
student_test_coef = {
    0.90: 1.643,
    0.95: 1.960,
    0.99: 2.580
}

def compute_characteristics_for_n_values(n):
    expected_value = sum(input_data[:n]) / n
    print(f"Оценка математического ожидания для {n} элементов: {expected_value}")

    dispersion = sum([(x - expected_value)**2 for x in input_data[:n]]) / (n - 1)
    print(f"Оценка диперсии для {n} элементов: {dispersion}")

    standard_deviation = math.sqrt(dispersion / len(input_data))
    print(f"Оценка среднеквадратического отклонения для {n} элементов: {standard_deviation}")
    
    epsilon_90 = student_test_coef[0.90] * standard_deviation
    print(f"Эпсилон 0.90 ({n} элементов): {epsilon_90}")
    
    epsilon_95 = student_test_coef[0.95] * standard_deviation
    print(f"Эпсилон 0.95 ({n} элементов): {epsilon_95}")
    
    epsilon_99 = student_test_coef[0.99] * standard_deviation
    print(f"Эпсилон 0.99 ({n} элементов): {epsilon_99}")
    
    return [expected_value, dispersion, standard_deviation, epsilon_90, epsilon_95, epsilon_99]
    
c_10 = compute_characteristics_for_n_values(10)
print()
c_50 = compute_characteristics_for_n_values(50)
print()
c_100 = compute_characteristics_for_n_values(100)
print()
c_200 = compute_characteristics_for_n_values(200)
print()
c_300 = compute_characteristics_for_n_values(300)

# TODO: print table

Оценка математического ожидания для 10 элементов: 1.920856418997817
Оценка диперсии для 10 элементов: 15.251124970522158
Оценка среднеквадратического отклонения для 10 элементов: 0.22547080350917692
Эпсилон 0.90 (10 элементов): 0.37044853016557766
Эпсилон 0.95 (10 элементов): 0.44192277487798676
Эпсилон 0.99 (10 элементов): 0.5817146730536765

Оценка математического ожидания для 50 элементов: 2.199855565563386
Оценка диперсии для 50 элементов: 15.765920806944548
Оценка среднеквадратического отклонения для 50 элементов: 0.2292445623269652
Эпсилон 0.90 (50 элементов): 0.3766488159032038
Эпсилон 0.95 (50 элементов): 0.4493193421608518
Эпсилон 0.99 (50 элементов): 0.5914509708035702

Оценка математического ожидания для 100 элементов: 1.9333170603657206
Оценка диперсии для 100 элементов: 14.603477111238902
Оценка среднеквадратического отклонения для 100 элементов: 0.22063149602326276
Эпсилон 0.90 (100 элементов): 0.36249754796622075
Эпсилон 0.95 (100 элементов): 0.432437732205595
Эпсилон 0.